<a href="https://colab.research.google.com/github/Lucasdz-7/Projeto-Integrador-2/blob/main/C%C3%B3digoAPi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
!apt-get install git-lfs -qq
import os
from google.colab import files

In [4]:
# Clonando o repositório do GitHub

repo_url = "https://github.com/Lucasdz-7/Projeto-Integrador-2.git"
repo_dir = "Projeto-Integrador-2"

if not os.path.exists(repo_dir):
    !git clone {repo_url}
    %cd {repo_dir}
else:
    %cd {repo_dir}
    !git pull

Cloning into 'Projeto-Integrador-2'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 9), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 14.66 KiB | 14.66 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/Projeto-Integrador-2


In [5]:
!git lfs install
!git lfs pull

Updated git hooks.
Git LFS initialized.


In [6]:
# Criando Dataframes

csv_path = "Dados Base/dados_ibama.csv"
Dados_IBAMA = pd.read_csv(csv_path, low_memory=False, sep=';', encoding='UTF-8')

In [12]:
Dados_IBAMA.head(10)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Quantidade Transportada,Unidade de Medida,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral
0,04.213.248/0001-91,TRANSDAG TRANSPORTES E SERVIÇOS LTDA,BAHIA,CATU,2008,"Álcool etílico não desnaturado, com teor alcoó...","147.528,00",Metro Cúbico,Rodoviário,Tanques,Sim,PERNAMBUCO,ESCADA,BAHIA,CANDEIAS,Suspenso para Averiguações - dados cadastrais ...
1,05.198.812/0001-07,KIRST TRR LTDA,MATO GROSSO,LUCAS DO RIO VERDE,2008,Gasóleo (óleo diesel),"1.122.000,00",Litro,Rodoviário,Tanques,Sim,MATO GROSSO,ALTO TAQUARI,MATO GROSSO,LUCAS DO RIO VERDE,Ativa
2,89.707.178/0001-80,TRANSPORTADORA KNIPHOFF LTDA,RIO GRANDE DO SUL,CRUZ ALTA,2001,Gasolina automotiva,"500.000,00",Litro,Rodoviário,Tanques,Sim,RIO GRANDE DO SUL,CRUZ ALTA,RIO GRANDE DO SUL,TRES PASSOS,Ativa
3,89.707.178/0001-80,TRANSPORTADORA KNIPHOFF LTDA,RIO GRANDE DO SUL,CRUZ ALTA,2001,Gasóleo (óleo diesel),"900.000,00",Litro,Rodoviário,Tanques,Sim,RIO GRANDE DO SUL,CRUZ ALTA,RIO GRANDE DO SUL,IJUI,Ativa
4,89.707.178/0001-80,TRANSPORTADORA KNIPHOFF LTDA,RIO GRANDE DO SUL,CRUZ ALTA,2002,Mentol (alcool de mentol),"350.000,00",Litro,Rodoviário,Tanques,Sim,RIO GRANDE DO SUL,CRUZ ALTA,RIO GRANDE DO SUL,CRUZ ALTA,Ativa
5,30.411.573/0001-79,TRANSPORTADORA NORTE FLUMINENSE DE MACAE LTDA,RIO DE JANEIRO,MACAE,2009,Óleos e produtos da destilação do alcatrão de ...,"10.000,00",Litro,Rodoviário,Tanques,Não,RIO DE JANEIRO,MACAE,ESPIRITO SANTO,VITORIA,Ativa
6,01.382.443/0001-57,MULTILIXO REMOÇÕES DE LIXO LTDA.,SAO PAULO,SAO PAULO,2008,óleo lubrificante usado,"12,00",Ton. por ano,Rodoviário,Outros,Sim,SAO PAULO,SAO PAULO,RIO DE JANEIRO,DUQUE DE CAXIAS,Ativa
7,51.013.233/0001-60,ARNOSTI TRANSPORTES LTDA,SAO PAULO,ARARAQUARA,2000,Gasóleo (óleo diesel),"139.396.650,00",Litro,Rodoviário,Outros,Sim,SAO PAULO,PAULINIA,SAO PAULO,BOTUCATU,Ativa
8,51.013.233/0001-60,ARNOSTI TRANSPORTES LTDA,SAO PAULO,ARARAQUARA,2002,Gasolina automotiva,"117.395.620,00",Litro,Rodoviário,Outros,Sim,SAO PAULO,PAULINIA,SAO PAULO,LEME,Ativa
9,51.013.233/0001-60,ARNOSTI TRANSPORTES LTDA,SAO PAULO,ARARAQUARA,2001,Fuel-oil (óleo combustível ),"60.735.222,00",Litro,Rodoviário,Outros,Sim,SAO PAULO,PAULINIA,SAO PAULO,INDAIATUBA,Ativa


In [8]:
Dados_IBAMA_2013a2023 = Dados_IBAMA[(Dados_IBAMA["Ano"] >= 2013) & (Dados_IBAMA["Ano"] <= 2023)]
Dados_IBAMA_2013a2023 = Dados_IBAMA_2013a2023.sort_values(by='Ano', ascending=True)

In [ ]:
Dados_IBAMA_2013a2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2941968 entries, 200354 to 281960
Data columns (total 16 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   CNPJ                     object
 1   Razão Social             object
 2   Estado                   object
 3   Município                object
 4   Ano                      int64 
 5   Produto                  object
 6   Quantidade Transportada  object
 7   Unidade de Medida        object
 8   Tipo de Transporte       object
 9   Tipo de Armazenamento    object
 10  Plano de Emergência      object
 11  UF - origem              object
 12  Município - origem       object
 13  UF - destino             object
 14  Município - destino      object
 15  Situação Cadastral       object
dtypes: int64(1), object(15)
memory usage: 381.6+ MB


In [9]:
Dados_IBAMA_2013a2023.head(5)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Quantidade Transportada,Unidade de Medida,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral
200354,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"30.951,00",Tonelada,Rodoviário,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa
200353,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"25.978,00",Tonelada,Rodoviário,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa
200352,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"13.190,00",Tonelada,Rodoviário,Tanques,Sim,SAO PAULO,COTIA,MINAS GERAIS,BELO HORIZONTE,Ativa
200288,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"34.830,00",Tonelada,Rodoviário,Tanques,Sim,SAO PAULO,COTIA,RIO DE JANEIRO,RESENDE,Ativa
200269,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"14.139,00",Tonelada,Rodoviário,Tanques,Sim,SAO PAULO,COTIA,MINAS GERAIS,UBERLANDIA,Ativa


In [10]:
Dados_IBAMA_2013a2023.tail(5)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Quantidade Transportada,Unidade de Medida,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral
3788372,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,"415,00",Metro Cúbico,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MANDAGUARI,Ativa
3787973,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,"Produtos inorgânicos, n.e, exceto os compostos...","603,00",Metro Cúbico,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MARINGA,Ativa
3788169,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Acetileno,"33,00",kilogramas,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,PATO BRANCO,Ativa
3788355,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,"190,00",Metro Cúbico,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,TUNEIRAS DO OESTE,Ativa
281960,09.446.066/0001-84,OXICAM GASES LTDA ME,SAO PAULO,PAULINIA,2023,Oxigênio,"30.000,00",kilogramas,Rodoviário,Tanques,Não,ESPIRITO SANTO,VITORIA,SAO PAULO,CAMPINAS,Ativa


In [14]:
Dados_IBAMA_teste = Dados_IBAMA_2013a2023.query("`Tipo de Transporte` != 'Rodoviário'")


In [20]:
Dados_IBAMA_teste.head(15)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Quantidade Transportada,Unidade de Medida,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral
225081,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),"25.812,00",Tonelada,Marítimo,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa
245153,04.965.146/0001-22,ROCHAFORTE TRANSPORTES E SERVICOS EIRELI,SAO PAULO,SOROCABA,2013,Cloreto de hidrogênio (ácido clorídrico),"72,00",Metro Cúbico,Marítimo,Tanques,Sim,SAO PAULO,TATUI,SAO PAULO,SUZANO,Ativa
252588,15.294.140/0001-97,V.B. DE ANDRADE TRANSPORTES ME,SAO PAULO,MIGUELOPOLIS,2013,Gás liqüefeito de petróleo (GLP),"22,00",Tonelada,Marítimo,Tanques,Sim,RIO DE JANEIRO,RIO DE JANEIRO,RIO DE JANEIRO,DUQUE DE CAXIAS,Encerrado
268263,04.901.176/0001-75,MB TRANSPORTES LTDA,SAO PAULO,SOROCABA,2013,Cloro,"27.000,00",kilogramas,Marítimo,Armazém Coberto,Sim,SAO PAULO,SOROCABA,SAO PAULO,SAO PAULO,Ativa
294573,06.046.590/0001-70,PALACIO COMERCIAL DE DERIVADOS DE PETROLEO LTDA,CEARA,CRATO,2013,Gasolina automotiva formulada,"200,00",Metro Cúbico,Ferroviário,Tanques,Sim,RIO GRANDE DO NORTE,NATAL,CEARA,CRATO,Ativa
295820,04.859.280/0001-49,CIA AGROLATINA DE ÁLCOOL E DERIVADOS LTDA,PARANA,QUEDAS DO IGUACU,2013,Álcool etílico desnaturado,"30,00",Caixa,Marítimo,Outros,Não,PARANA,QUEDAS DO IGUACU,RIO GRANDE DO SUL,CAIBATE,Ativa
296869,42.278.291/0001-24,LOG-IN - LOGISTICA INTERMODAL S/A,RIO DE JANEIRO,RIO DE JANEIRO,2013,Hipocloritos de sódio (exceto água sanitária),"30,00",Tonelada,Marítimo,Outros,Não,ESPIRITO SANTO,SERRA,AMAZONAS,MANAUS,Ativa
799575,01.236.287/0001-16,AGRO 100 COMERCIO DE PRODUTOS AGRICOLAS LTDA -...,PARANA,LONDRINA,2013,Metamidofós; dissulfotom; tiometon; acefato (p...,"180,00",Litro,Marítimo,Armazém Coberto,Sim,PARANA,LONDRINA,PARANA,CAFEARA,Ativa
303215,42.278.291/0001-24,LOG-IN - LOGISTICA INTERMODAL S/A,RIO DE JANEIRO,RIO DE JANEIRO,2013,Álcool propílico (propan - 1 - ol; álcool isop...,"154,00",Tonelada,Marítimo,Outros,Não,SAO PAULO,GUARULHOS,CEARA,MARACANAU,Ativa
303310,42.278.291/0001-24,LOG-IN - LOGISTICA INTERMODAL S/A,RIO DE JANEIRO,RIO DE JANEIRO,2013,"Produtos químicos, n.e.","29,00",Tonelada,Marítimo,Outros,Não,BAHIA,SIMOES FILHO,AMAZONAS,MANAUS,Ativa
